# GPT Action Library - Adzviser経由でのGoogle Ads

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：
- [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
- [GPT Actionsライブラリの紹介](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

このガイドでは、**Google Ads**のレポートデータをChatGPTに接続して、キャンペーン、広告グループ、または広告レベルでのインプレッション、クリック、コストなどの主要なパフォーマンス指標を取得する方法を説明します。このプロセスを簡素化するために、[Adzviser](https://adzviser.com)をミドルウェアとして使用します。これにより、Google Ads APIから返されるデータが適切にフォーマットされ、ChatGPTの[データ分析](https://help.openai.com/en/articles/8437071-data-analysis-with-chatgpt)環境での分析に対応できるようになります。

**Adviserの仕組み：** まず、[OAuth](https://docs.adzviser.com/getStarted/workspace)を通じてGoogle AdsアカウントをAdzviserに接続します。ChatGPTで「先月キャンペーンごとにいくら使ったか？」などの質問をすると、Adviserが[Google Ads Query Language](https://developers.google.com/google-ads/api/docs/query/overview)リクエストを送信し、レスポンスをCSVファイル（10MB未満）に変換します。このファイルは[ChatGPTに返され](https://platform.openai.com/docs/actions/sending-files/returning-files)、分析に使用されます。Adviserを使用することで、過去のデータインサイトに基づいて最適化戦略を検討しながら、キャンペーンのパフォーマンスを簡単にレビューし分析できるようになります。

### 価値 + ビジネス活用事例

**価値**: Google Adsマーケターは、ChatGPTの自然言語機能を活用して、Google Ads UIをナビゲートすることなく、パフォーマンス指標やアカウント設定を簡単にクエリできるようになりました。プロセス全体でファイルをアップロードまたはダウンロードする必要はありません。

**使用例**: 
- eコマースビジネスオーナーが、前月のGoogle Adsキャンペーンの広告費用対効果（ROAS）を素早く確認したい場合
- ブランドマーケターが、過去3か月のレポートデータを使用してキーワードと検索語句の分析を行い、どのキーワードを停止または拡大するか、どの検索語句を除外キーワードとして追加するかを特定することを目的とする場合
- 代理店マーケターが、クリック単価（CPC）、コンバージョン単価（CPA）、検索インプレッションシェアなどの主要指標を含む月次レポートを、前月比較とともに生成する必要がある場合
- フリーランスマーケターが、オンボーディングプロセス中に新しいクライアントのGoogle Adsアカウントを監査して、パフォーマンスを評価し、最適化の機会を見つける必要がある場合

## デモ/例

![GPT検索語句分析 パート1](../../../images/gptactions_googleads_search_term_analysis_1.png)![GPT検索語句分析 パート2](../../../images/gptactions_googleads_search_term_analysis_2.png)

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションから以下のリンクをご確認ください：
- Adzviserでワークスペースを作成する方法: https://docs.adzviser.com/getStarted/workspace
- Adzviser Custom GPTドキュメント: https://docs.adzviser.com/chatgpt/expert
- Google Adsプロンプトライブラリ: https://docs.adzviser.com/chatgpt/googleAdsPromptTemplates

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- Google Adsアカウントに対して[読み取り専用、標準、または管理者アクセス](https://support.google.com/google-ads/answer/9978556?hl=en)があることを確認する
- Adzviserで[アカウントにサインアップ](https://adzviser.com/signup)し、サブスクリプションを有効化する（$0.99から開始）
- ワークスペースを作成してGoogle AdsアカウントをAdzviserに接続する

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
***Context***:
You are a Google Ads specialist who audits account health, retrieves real-time reporting data, and optimizes performances for marketers. When asked for an audit on account health, collect the relevant account settings, provide recommendations to adjust account structures. When asked about reporting data insights, gather relevant metrics and breakdowns, thoroughly analyze the reporting data, and then provide tailored recommendations to optimize performance.

***Instructions for Retrieval of Reporting Data***:
- Workflow to fetch real-time reporting data
Step 1. Calculate the date range with Python and Code Interpreter based on user input, such as "last week", "last month", "yesterday", "last 28 days", "last quarter" or "last year" etc. If no specific timeframe is provided, ask the user to clarify. Adjust for calendar variations. For example, "last week" should cover Monday to Sunday of the previous week. 
Step 2. Retrieve workspace information using the 'getWorkspace' function. 
Step 3. Fetch the relevant metrics and breakdowns for the inquired data source using functions like 'getGoogleAdsMetricsList' and 'getGoogleAdsBreakdownsList'.
Step 4. Use 'searchQuery' function with the data gathered from the previous steps like available workspace_name and metrics/breakdowns as well as calculated date range to retrieve real-time reporting data.
- Time Granularity: If the user asks for daily/weekly/quarterly/monthly data, please reflect such info in the field time_granularity in searchQueryRequest. No need to add time_granularity if the user did not ask for it explicitly.
- Returned Files: If multiple files are returned, make sure to read all of them. Each file contains data from a segment in a data source or a data source. 
- Necessary Breakdowns Only: Add important breakdowns only. Less is more. For example, if the user asks for "which ad is performing the best in Google Ads?", then you only add "Ad Name" in the breakdown list for the google_ads_request. No need to add breakdowns such as "Device" or "Campaign Name". 

***Instruction for Auditing****:
- Workflow to audit Google Ads account
Step 1. Retrieve workspace information using the 'getWorkspace' function.
Step 2. Use '/google_ads_audit/<specfic_section_to_check>' function to retrieve account settings.
- Comprehensive Audit: When asked for an comprehensive audit, don't call all the /google_ads_audit/<specfic_section_to_check> all at once. Show the users what you're planning to do next first. Then audit two sections from the Google Ads Audit GPT Knowledge at a time, then proceed to the next two sections following users consent. For the line items in the tables in the Audit Knowledge doc that don't have automation enabled, it is very normal and expected that no relevant data is seen in the retrieved response. Please highlight what needs to be checked by the user manually because these non-automated steps are important too. For example, when checking connections, adzviser only checks if the google ads account is connected with Google Merchant Center. For other connections such as YT channels, please politely ask the user to check them manually. 

***Additional Notes***:
- Always calculate the date range please with Code Interpreter and Python. It often is the case that you get the date range 1 year before when the user asks for last week, last month, etc. 
- If there is an ApiSyntaxError: Could not parse API call kwargs as JSON, please politely tell the user that this is due to the recent update in OpenAI models and it can be solved by starting a new conversation on ChatGPT.
- If the users asks for Google Ads data, for example, and there is only one workspace that has connected to Google Ads, then use this workspace name in the searchQueryRequest or googleAdsAuditRequest.
- During auditing, part of the process is to retrieve the performance metrics at account, campaign, ad group, keyword, and product levels, remember to also run Python to calculate the date range for last month and the previous period. For retrieving performance metrics at these 5 levels, please send 5 distinct requests with different breakdowns list for each level. More can be found in the audit knowledge doc.

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
{
  "openapi": "3.1.0",
  "info": {
    "title": "Adzviser Actions for GPT",
    "description": "Equip GPTs with the ability to retrieve real-time reporting data and account settings from Google Ads",
    "version": "v0.0.1"
  },
  "servers": [
    {
      "url": "https://copter.adzviser.com"
    }
  ],
  "paths": {
    "/google_ads/get_metrics_list": {
      "get": {
        "description": "Get the list of seletable Google Ads metrics, such as Cost, Roas, Impressions, etc.",
        "operationId": "getGoogleAdsMetricsList",
        "parameters": [],
        "deprecated": false,
        "security": [],
        "x-openai-isConsequential": false
      }
    },
    "/google_ads/get_breakdowns_list": {
      "get": {
        "description": "Get the list of seletable Google Ads breakdowns such as Device, Keyword Text, Campaign Name etc.",
        "operationId": "getGoogleAdsBreakdownsList",
        "parameters": [],
        "deprecated": false,
        "security": [],
        "x-openai-isConsequential": false
      }
    },
    "/search_bar": {
      "post": {
        "description": "Retrieve real-time reporting data such as impressions, cpc, etc. from marketing channels such as Google Ads, Fb Ads, Fb Insights, Bing Ads, etc.",
        "operationId": "searchQuery",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/searchQueryRequest"
              }
            }
          },
          "required": true
        },
        "deprecated": false,
        "security": [
          {
            "oauth2": []
          }
        ],
        "x-openai-isConsequential": false
      }
    },
    "/workspace/get": {
      "get": {
        "description": "Retrieve a list of workspaces that have been created by the user and their data sources, such as Google Ads, Facebook Ads accounts connected with each.",
        "operationId": "getWorkspace",
        "parameters": [],
        "deprecated": false,
        "security": [
          {
            "oauth2": []
          }
        ],
        "responses": {
          "200": {
            "description": "OK",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/getWorkspaceResponse"
                }
              }
            }
          }
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_merchant_center_connection": {
      "post": {
        "description": "Retrieve whether the Google Merchant Center is connected to the Google Ads account.",
        "operationId": "checkGoogleAdsMerchantCenterConnection",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_account_settings": {
      "post": {
        "description": "Retrieve the Google Ads account settings such as whether auto tagging is enabled, inventory type, etc.",
        "operationId": "checkGoogleAdsAccountSettings",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_negative_keywords_and_placements": {
      "post": {
        "description": "Retrieve the negative keywords and placements set in the Google Ads account.",
        "operationId": "checkGoogleAdsNegativeKeywordsAndPlacements",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_remarketing_list": {
      "post": {
        "description": "Retrieve the remarketing list set in the Google Ads account.",
        "operationId": "checkGoogleAdsRemarketingList",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_conversion_tracking": {
      "post": {
        "description": "Retrieve the conversion tracking status in the Google Ads account.",
        "operationId": "checkGoogleAdsConversionTracking",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_bidding_strategy": {
      "post": {
        "description": "Retrieve the bidding strategy set for each active campaigns in the Google Ads account.",
        "operationId": "checkGoogleAdsBiddingStrategy",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_search_campaign_basic": {
      "post": {
        "description": "Retrieve the basic information of the search campaigns such as campaign structure, language targeting, country targeting, etc.",
        "operationId": "checkSearchCampaignBasic",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_search_campaign_detailed": {
      "post": {
        "description": "Retrieve the detailed information of the search campaigns such as best performing keywords, ad copies, ad extentions, pinned descriptions/headlines etc.",
        "operationId": "checkSearchCampaignDetailed",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_dynamic_search_ads": {
      "post": {
        "description": "Retrieve the dynamic search ads information such as dynamic ad targets, negative ad targets, best performing search terms etc.",
        "operationId": "checkDynamicSearchAds",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    },
    "/google_ads_audit/check_pmax_campaign": {
      "post": {
        "description": "Retrieve the performance of the pmax campaigns such as search themes, country/language targeting, final url expansions, excluded urls.",
        "operationId": "checkPmaxCampaign",
        "parameters": [],
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/googleAdsAuditRequest"
              }
            }
          },
          "required": true
        },
        "x-openai-isConsequential": false
      }
    }
  },
  "components": {
    "schemas": {
      "getWorkspaceResponse": {
        "title": "getWorkspaceResponse",
        "type": "array",
        "description": "The list of workspaces created by the user on adzviser.com/main. A workspace can include multiple data sources",
        "items": {
          "type": "object",
          "properties": {
            "name": {
              "title": "name",
              "type": "string",
              "description": "The name of a workspace"
            },
            "data_connections_accounts": {
              "title": "data_connections_accounts",
              "type": "array",
              "description": "The list of data sources that the workspace is connected. The name can be an account name and type can be Google Ads/Facebook Ads/Bing Ads",
              "items": {
                "type": "object",
                "properties": {
                  "name": {
                    "title": "name",
                    "type": "string",
                    "description": "The name of a data connection account"
                  }
                }
              }
            }
          }
        }
      },
      "googleAdsAuditRequest": {
        "description": "Contains details about the Google Ads account audit request.",
        "type": "object",
        "required": [
          "workspace_name"
        ],
        "title": "googleAdsAuditRequest",
        "properties": {
          "workspace_name": {
            "type": "string",
            "title": "workspace_name",
            "description": "Call API getWorkspace first to get a list of available workspaces"
          }
        }
      },
      "searchQueryRequest": {
        "description": "Contains details about queried data source, metrics, breakdowns, time ranges and time granularity, etc.",
        "type": "object",
        "required": [
          "assorted_requests",
          "workspace_name",
          "date_ranges"
        ],
        "title": "searchQueryRequest",
        "properties": {
          "assorted_requests": {
            "type": "object",
            "title": "assorted_requests",
            "description": "For example, if the user asks for \"cost on Google ads last month\", then call getGoogleAdsMetricsList and getGoogleAdsBreakdownsList to retrieve the latest up-to-date info about how to compose a google_ads_request. A metric is a quantitative measurement. It represents data that can be measured and expressed in numbers. Metrics are used to track performance or behavior. Examples include clicks, impressions, conversions, revenue, etc. A breakdown is a qualitative attribute or descriptor. It provides context for metrics by categorizing or segmenting them. Breakdowns are text. Examples include country, channel, campaign name, etc. DO NOT include Date, Month, Quarter or Year in the list of breakdowns in any of the requests below. The breakdowns should be NOT mixed up with metrics, meaning that the selected breakdowns should be passed into the property \"breakdowns\", not \"metrics\", and vice versa.",
            "properties": {
              "google_ads_request": {
                "type": "object",
                "description": "DO NOT come up with metrics and breakdowns on your own. You MUST call API getGoogleAdsMetricsList and getGoogleAdsBreakdownsList to be better informed prior of composing a googleAdsRequest.",
                "required": [
                  "metrics",
                  "breakdowns"
                ],
                "properties": {
                  "breakdowns": {
                    "type": "array",
                    "items": {
                      "type": "string"
                    },
                    "description": "Must call API getGoogleAdsBreakdownsList to retrieve a list of selectable breakdowns."
                  },
                  "metrics": {
                    "type": "array",
                    "items": {
                      "type": "string"
                    },
                    "description": "Must call API getGoogleAdsMetricsList to retrieve a list of selectable metrics."
                  }
                }
              }
            }
          },
          "workspace_name": {
            "type": "string",
            "title": "workspace_name",
            "description": "Call API getWorkspace first to get a list of available workspaces. Multiple data sources (such as Google ads, Bing ads) can be stored in one workspace. If the user does not specify a workspace name, then use the available workspace name from the retrieved list and see which one has Google Ads. If the user has not yet created one, then ask them to go to adzviser.com/main to create a new workspace."
          },
          "date_ranges": {
            "type": "array",
            "description": "A list of date ranges requested from the user. They needs to be calculated seperately with Code Interpreter and Python every single time for accuracy. For example, if the user requests \"Google Ads search impression share in May and August\", then this array should be [[\"2024-05-01\", \"2024-05-31\"], [\"2024-08-01\", \"2024-08-31\"]].",
            "items": {
              "type": "array",
              "items": {
                "type": "string"
              },
              "description": "A 2-element array. The first represents the start date and the second the end date. Both are in YYYY-MM-DD format."
            }
          },
          "time_granularity": {
            "type": "string",
            "title": "time_granularity",
            "default": "",
            "description": "Describes how granularity you wish the date_ranges to be. For example, If the user asks \"weekly cost on Google Ads\" this year, then this value should be \"Week\". If the user does not specify, then leave it as empty.",
            "enum": [
              "Date",
              "Week",
              "Month",
              "Quarter"
            ]
          }
        }
      }
    },
    "securitySchemes": {
      "oauth2": {
        "type": "oauth2"
      }
    }
  }
}

## 認証手順

以下は、このサードパーティアプリケーションでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

![GPT OAuth設定](../../../images/gptactions_adzviser_oauth.png)

### ChatGPTにおいて

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択してください。以下の情報を入力してください。

- **Client ID**: （空白のまま）
- **Client Secret**: （空白のまま）
- **Authorization URL**: https://adzviser.com/authorize-gpt
- **Token URL**: https://adzviser.com/api/oauth-exchange-token-gpt
- **Scope**: （空白のまま）
- **Token Exchange Method**: Default (POST)

### FAQ & トラブルシューティング

- *空のGoogle Ads アカウントリスト*: Google Ads アカウントを接続しようとした際に空のGoogle Ads アカウントリストが表示される場合、まだGoogle Ads アカウントに名前を付けていない可能性があります。これを解決するには、ads.google.com にアクセスしてサインインしてください。その後、[こちら](https://support.google.com/google-ads/answer/7519527?hl=en)の手順に従ってGoogle Ads アカウントに名前を付けてください。

*優先的に対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*